In [4]:
import csv
import random
import gzip
import shutil

def create_masked_csv(
    separator: str = ":",
):

    with gzip.open("../data/comp_1.gz", "rb") as f_in:
        with open("../data/comp_1.csv", "wb") as f_out:
            shutil.copyfileobj(f_in, f_out)


    with open("../data/comp_1.csv", "r", newline="", encoding="utf-8") as infile:
        reader = csv.DictReader(infile)
        rows = list(reader)


    # Split the dataset into validation (20%) and training (80%)
    split_idx = len(rows) // 5
    all_rows = []
    for row in rows:
        all_rows.extend([row] * int(row['Redundancy']))
    rows = all_rows
    random.shuffle(rows)

    validation_rows = rows[:split_idx]
    training_rows = rows[split_idx:]

    fieldnames = ["id", "sequence"]

    # Write validation file
    with open("../data/val.csv", "w", newline="", encoding="utf-8") as val_outfile:
        writer = csv.DictWriter(val_outfile, fieldnames=fieldnames)
        writer.writeheader()
        for row_id, row in enumerate(validation_rows):
            writer.writerow({
                "id": f"antibody_{row_id}",
                "sequence": f"{row['sequence_aa_heavy']}{separator}{row['sequence_aa_light']}"
            })

    # Write training file
    with open("../data/train.csv", "w", newline="", encoding="utf-8") as train_outfile:
        writer = csv.DictWriter(train_outfile, fieldnames=fieldnames)
        writer.writeheader()
        for row_id, row in enumerate(training_rows, start=len(validation_rows)):
            writer.writerow({
                "id": f"antibody_{row_id}",
                "sequence": f"{row['sequence_aa_heavy']}{separator}{row['sequence_aa_light']}"
            })

# Run the function
create_masked_csv(separator=":")
